## Now, we will train our model

- We will use VGG16 as a pretrained classifer 
- By fine-tuning we will use VGG16 as our classifier

### Import Libraries

In [1]:
import os
import numpy as np
from keras.layers import *
from keras.preprocessing import image
from keras.applications import VGG16, MobileNetV2, ResNet50
from keras.models import Model
from keras.optimizers import Adam

### Create Data-Generator

In [2]:
path = '../Datasets/Facial Expressions/'

In [3]:
train_gen = image.ImageDataGenerator(rescale=1.0/255)
val_gen = image.ImageDataGenerator(rescale=1.0/255)

train_generator = train_gen.flow_from_directory(
    path+'train/',
    target_size=(150,150),
    batch_size=64,
    class_mode='categorical'
)

val_generator = val_gen.flow_from_directory(
    path+'val/',
    target_size=(150,150),
    batch_size=64,
    class_mode='categorical'
)

Found 5180 images belonging to 8 classes.
Found 1292 images belonging to 8 classes.


### Load VGG16 Model

In [4]:
model = VGG16(include_top=False, input_shape=(150,150,3))

av1 = GlobalAveragePooling2D()(model.output)
f1 = Flatten()(av1)
drop1 = Dropout(0.5)(f1)
d1 = Dense(256,activation='relu')(drop1)
d2 = Dropout(0.5)(d1)
out = Dense(8,activation='softmax')(d2)

In [5]:
vgg = Model(model.input,out)
print(len(vgg.layers))

25


In [6]:
for i in range(17): 
    vgg.layers[i].trainable = False
vgg.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)      

In [7]:
adam = Adam(lr=0.001)
vgg.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

- Train Model

In [8]:
hist_vgg = vgg.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

Epoch 1/20
57/57 [==============================] - 880s 15s/step - loss: 1.9467 - accuracy: 0.2593 - val_loss: 1.5704 - val_accuracy: 0.4564
Epoch 2/20
57/57 [==============================] - 866s 15s/step - loss: 1.5191 - accuracy: 0.4315 - val_loss: 1.2675 - val_accuracy: 0.5469
Epoch 3/20
57/57 [==============================] - 868s 15s/step - loss: 1.2902 - accuracy: 0.5353 - val_loss: 1.0718 - val_accuracy: 0.6571
Epoch 4/20
57/57 [==============================] - 875s 15s/step - loss: 1.1238 - accuracy: 0.5941 - val_loss: 0.9668 - val_accuracy: 0.6847
Epoch 5/20
57/57 [==============================] - 875s 15s/step - loss: 0.9824 - accuracy: 0.6572 - val_loss: 0.8959 - val_accuracy: 0.6880
Epoch 6/20
57/57 [==============================] - 875s 15s/step - loss: 0.8735 - accuracy: 0.6949 - val_loss: 0.7635 - val_accuracy: 0.7310
Epoch 7/20
57/57 [==============================] - 876s 15s/step - loss: 0.7843 - accuracy: 0.7333 - val_loss: 0.7507 - val_accuracy: 0.7630
Epoch 

### Load MobilenetV2 Model

In [4]:
model = MobileNetV2(include_top=False, input_shape=(128,128,3))

av1 = GlobalAveragePooling2D()(model.output)
f1 = Flatten()(av1)
drop1 = Dropout(0.5)(f1)
d1 = Dense(256,activation='relu')(drop1)
d2 = Dropout(0.5)(d1)
out = Dense(8,activation='softmax')(d2)

In [5]:
mobile = Model(model.input,out)
print(len(mobile.layers))

161


In [6]:
for i in range(152): 
    mobile.layers[i].trainable = False
mobile.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________________

block_4_project_BN (BatchNormal (None, 16, 16, 32)   128         block_4_project[0][0]            
__________________________________________________________________________________________________
block_4_add (Add)               (None, 16, 16, 32)   0           block_3_project_BN[0][0]         
                                                                 block_4_project_BN[0][0]         
__________________________________________________________________________________________________
block_5_expand (Conv2D)         (None, 16, 16, 192)  6144        block_4_add[0][0]                
__________________________________________________________________________________________________
block_5_expand_BN (BatchNormali (None, 16, 16, 192)  768         block_5_expand[0][0]             
__________________________________________________________________________________________________
block_5_expand_relu (ReLU)      (None, 16, 16, 192)  0           block_5_expand_BN[0][0]          
__________

Total params: 2,587,976
Trainable params: 742,152
Non-trainable params: 1,845,824
__________________________________________________________________________________________________


In [7]:
adam = Adam(lr=0.001)
mobile.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

- Train Mobilenetv2 Model

In [8]:
hist_mobile = mobile.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

Epoch 1/20
57/57 [==============================] - 185s 3s/step - loss: 1.6944 - accuracy: 0.4320 - val_loss: 1.5550 - val_accuracy: 0.4653
Epoch 2/20
57/57 [==============================] - 57s 1s/step - loss: 1.0588 - accuracy: 0.6358 - val_loss: 1.3352 - val_accuracy: 0.5502
Epoch 3/20
57/57 [==============================] - 55s 973ms/step - loss: 0.8432 - accuracy: 0.7171 - val_loss: 1.1671 - val_accuracy: 0.6196
Epoch 4/20
57/57 [==============================] - 55s 970ms/step - loss: 0.7175 - accuracy: 0.7509 - val_loss: 1.4813 - val_accuracy: 0.5469
Epoch 5/20
57/57 [==============================] - 61s 1s/step - loss: 0.6278 - accuracy: 0.7841 - val_loss: 1.2311 - val_accuracy: 0.6262
Epoch 6/20
57/57 [==============================] - 64s 1s/step - loss: 0.5397 - accuracy: 0.8127 - val_loss: 1.0728 - val_accuracy: 0.6814
Epoch 7/20
57/57 [==============================] - 75s 1s/step - loss: 0.5112 - accuracy: 0.8190 - val_loss: 1.2592 - val_accuracy: 0.6461
Epoch 8/20
57

### Load ResNet50 Model

In [9]:
model = ResNet50(include_top=False, input_shape=(150,150,3))

av1 = GlobalAveragePooling2D()(model.output)
f1 = Flatten()(av1)
drop1 = Dropout(0.5)(f1)
d1 = Dense(256,activation='relu')(drop1)
d2 = Dropout(0.5)(d1)
out = Dense(8,activation='softmax')(d2)

In [10]:
resnet = Model(model.input,out)
print(len(resnet.layers))

181


In [11]:
for i in range(169):
    resnet.layers[i].trainable=False
resnet.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
_______________________________________________________________________________________

__________________________________________________________________________________________________
conv4_block4_2_relu (Activation (None, 10, 10, 256)  0           conv4_block4_2_bn[0][0]          
__________________________________________________________________________________________________
conv4_block4_3_conv (Conv2D)    (None, 10, 10, 1024) 263168      conv4_block4_2_relu[0][0]        
__________________________________________________________________________________________________
conv4_block4_3_bn (BatchNormali (None, 10, 10, 1024) 4096        conv4_block4_3_conv[0][0]        
__________________________________________________________________________________________________
conv4_block4_add (Add)          (None, 10, 10, 1024) 0           conv4_block3_out[0][0]           
                                                                 conv4_block4_3_bn[0][0]          
__________________________________________________________________________________________________
conv4_bloc

In [12]:
adam=Adam(lr=0.005)
resnet.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

- Train Model

In [13]:
hist_resnet = resnet.fit(
    train_generator,
    epochs=3,
    validation_data=val_generator
)

Epoch 1/3
81/81 [==============================] - 578s 7s/step - loss: 2.1540 - accuracy: 0.1371 - val_loss: 2.0666 - val_accuracy: 0.1827
Epoch 2/3
81/81 [==============================] - 601s 7s/step - loss: 2.0225 - accuracy: 0.1803 - val_loss: 2.0292 - val_accuracy: 0.1981
Epoch 3/3
81/81 [==============================] - 577s 7s/step - loss: 1.9937 - accuracy: 0.1880 - val_loss: 2.0222 - val_accuracy: 0.2113
